<a href="https://colab.research.google.com/github/PeLaNo2/Petchsaritpong/blob/main/Research_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os
os.environ["GEMINI_API_KEY"]=userdata.get('gemini_api')

In [ ]:
import csv
import os
from google import genai
from google.genai import types
import time
import re
from tqdm import tqdm


def get_company_address(Company : str) -> str:
    """
    Generates company address and email using Gemini with Google Search.
    Returns the address as a string or an empty string if not found/error.
    """
    if not Company :
        return ""
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"

    text_prompt = f"""
Search for the full official address and Email of the company named "{Company}" using Google Maps or Google Search.
Provide the complete address including street number, street name, sub-district (tambon/khwaeng), district (amphoe/khet), province (changwat), and postal code.
 and also email. The address should be in English.

Reason step by step, then provide the best matching full address within a <final_answer> tag.
For example: <final_answer>123 Silom Road, Silom, Bang Rak, Bangkok, 10500, Thailand</final_answer>
Another example: <final_answer>999/9 Rama I Road, Pathum Wan, Pathum Wan District, Bangkok 10330, Thailand</final_answer>
If you cannot find a definitive address, output <final_answer>Not Found</final_answer>.
Output ONLY the content for the <final_answer> tag.
"""

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=text_prompt ),
            ],
        ),
    ]

    tools = [
        types.Tool(google_search=types.GoogleSearch()),
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=0.1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        tools=tools,
        response_mime_type="text/plain",
    )

    try:
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=generate_content_config,
        )

        match = re.search(r"<final_answer>(.*?)</final_answer>", response.text)
        if match:
            final_answer = match.group(1).strip()
            new_item_numbers = []
            for num_str in final_answer.split(','):
                num_str = num_str.strip()
                if num_str.isdigit():
                    new_item_numbers.append(num_str)

            return new_item_numbers

        else:
            print(f"Warning: <final_answer> tag not found.")
            return []

    except Exception as e:
        print(f"Error during Gemini inference: {e}")
        return []


# Correct Indentation:
def process_csv(input_file, output_file, start_row=1, end_row=None):
    """Processes the CSV, prioritizing existing items, adding new (up to 5)."""
    try: # Good practice to wrap file operations in try/except
        with open(input_file, 'r', encoding='cp1252') as infile, \
                open(output_file, 'w', newline='', encoding='utf-8') as outfile:

            # ---> START: All these lines MUST be indented under 'with' <---
            reader = csv.DictReader(infile)

            # Prepare fieldnames for the writer
            fieldnames = reader.fieldnames
            if fieldnames is None:
                 print(f"ERROR: Could not read headers from {input_file}. Is it empty or not a valid CSV?")
                 return # Exit if no headers

            # Check if 'ITEM' column exists, if not, add it to the list for the writer
            if 'CATEGORIES NO' not in fieldnames:
                fieldnames = fieldnames + ['CATEGORIES NO'] # Create a new list including 'CATEGORIES NO'

            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader() # Write the potentially modified headers

            # Wrap the reader with tqdm *here*, and use enumerate on tqdm(reader)
            # Note: total=end_row might be inaccurate if end_row is beyond file length
            # Consider removing 'total' or doing a first pass to count lines for accuracy
            print(f"Processing rows from {start_row}...") # Add some logging
            for row_count, row in tqdm(enumerate(reader, start=1), desc="Processing CSV"):
                if row_count < start_row:
                    # Need to write skipped rows too, ensuring all columns are present
                    output_row = {field: row.get(field, '') for field in fieldnames}
                    if 'CATEGORIES NO' not in row: output_row['CATEGORIES NO'] = '' # Ensure ITEM exists for writer
                    writer.writerow(output_row)
                    continue
                if end_row is not None and row_count > end_row:
                    break # Stop processing after end_row

                # --- Your row processing logic would go here ---
                # Example placeholder:
                Company = row.get('Company ', '').strip()
                PRODUCT_CODE = row.get('PRODUCT CODE', '').strip()
                product_category = row.get('PRODUCT CATEGORY', '').strip()
                existing_items_str = row.get('CATEGORIES NO', '').strip() # Get existing CATEGORIES NO if column was present
                existing_items = [item.strip() for item in existing_items_str.split(',') if item.strip().isdigit()]

                if not product_category:
                  product_category = "Not specified"

                print(Company)

                if not Company:
                     print(f"\nSkipping row {row_count} due to missing data.")
                     output_row = {field: row.get(field, '') for field in fieldnames}
                     if 'CATEGORIES NO' not in row: output_row['CATEGORIES NO'] = existing_items_str # Write original ITEM if present
                     elif not output_row.get('CATEGORIES NO'): output_row['CATEGORIES NO'] = '' # Ensure ITEM exists
                     writer.writerow(output_row)
                     continue

                time.sleep(4.1) # Consider moving delay right before API call if possible

                new_items = generate_items(Company, PRODUCT_CODE, product_category, existing_items) # Call your Gemini function
                combined_items = existing_items.copy()
                for item in new_items:
                    if item.isdigit() and item not in combined_items and len(combined_items) < 5:
                        combined_items.append(item)

                final_items = [item for item in combined_items[:5] if item.isdigit()]

                # Prepare the row for writing, ensuring all fields exist
                output_row = {field: row.get(field, '') for field in fieldnames}
                output_row['ADD1'] = ", ".join(final_items) # Add the processed items

                writer.writerow(output_row)
                # ---> END: Indentation under 'with' ends here <---

    except FileNotFoundError:
        print(f"ERROR: Input file not found: {input_file}")
    except Exception as e:
        print(f"An error occurred during CSV processing: {e}")

if __name__ == "__main__":
    input_csv_file = "/content/複本 The Inspired Home Show 2025_NON-US_BUYERS_LIST(1)(moke).csv"  # Your input CSV file
    output_csv_file = "output.csv"
    start_row = 500
    end_row = 505  # Or None to process to the end
    process_csv(input_file=input_csv_file, output_file=output_csv_file, start_row=start_row, end_row=end_row)
    print("Processing complete. Results saved to:", output_csv_file)


An error occurred during CSV processing: 'charmap' codec can't decode byte 0x8d in position 5510: character maps to <undefined>
Processing complete. Results saved to: output.csv
